### Import model

In [1]:
import torch
import torchvision

/Users/jvjv/opt/anaconda3/envs/ConSens/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from inference.models.alexnet import AlexnetMap_v2

In [3]:
grasp_model = AlexnetMap_v2()
cls_model = AlexnetMap_v2()

/Users/jvjv/opt/anaconda3/envs/ConSens/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/jvjv/opt/anaconda3/envs/ConSens/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
cls_model.load_state_dict(torch.load("trained-models/alexnetMap_cls_top5_v2.3_epoch150.pth", map_location=torch.device('cpu')))
grasp_model.load_state_dict(torch.load("trained-models/alexnetMap_grasp_top5_v2.1_epoch150.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [5]:
from evaluation import get_cls_acc, get_grasp_acc

In [6]:
get_cls_acc(cls_model)

(83.83, 0.0)

In [ ]:
get_grasp_acc(cls_model)

In [ ]:
get_cls_acc(grasp_model)

In [ ]:
get_grasp_acc(grasp_model)

### Neuron Shapley

In [7]:
def cls_criterion(model):
    return get_cls_acc(model)[0]

def grasp_criterion(model):
    return get_grasp_acc(model)[0]

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
for name, W in cls_model.named_parameters():
    print(name, W.shape)

rgb_features.0.weight torch.Size([64, 3, 11, 11])
rgb_features.0.bias torch.Size([64])
features.0.weight torch.Size([192, 128, 5, 5])
features.0.bias torch.Size([192])
features.4.weight torch.Size([384, 192, 3, 3])
features.4.bias torch.Size([384])
features.7.weight torch.Size([256, 384, 3, 3])
features.7.bias torch.Size([256])
features.10.weight torch.Size([256, 256, 3, 3])
features.10.bias torch.Size([256])
features.13.weight torch.Size([256, 128, 3, 3])
features.13.bias torch.Size([128])
features.15.weight torch.Size([128, 128, 5, 5])
features.15.bias torch.Size([128])
features.17.weight torch.Size([128, 64, 3, 3])
features.17.bias torch.Size([64])
grasp.0.weight torch.Size([64, 5, 11, 11])
grasp.0.bias torch.Size([5])
confidence.0.weight torch.Size([64, 1, 11, 11])
confidence.0.bias torch.Size([1])


In [21]:
from shapley import layer_wise_tmab_shapley as tmab
from shapley import layer_wise_tmc_shapley as tmc

In [23]:
shapley_score_cls = tmc(model=cls_model, 
                   layer="rgb_features.0.weight",
                   criterion=cls_criterion, 
                   iterations=10, 
                   vt=20, 
                   verbose=True)


Iteration 1/10:


  2%|▏         | 1/64 [00:24<25:38, 24.42s/it]

[82.0, 82.33]


  3%|▎         | 2/64 [00:46<24:06, 23.33s/it]

[82.0, 82.33, 82.67]


...

[82.0, 82.33, 82.67, 82.0, 78.17, 75.17, 74.5, 76.5, 74.33, 66.5, 64.5, 57.0, 58.33, 56.67, 59.0, 50.0, 47.83, 45.5, 48.83, 48.67]


 31%|███▏      | 20/64 [07:42<17:05, 23.30s/it]

[82.0, 82.33, 82.67, 82.0, 78.17, 75.17, 74.5, 76.5, 74.33, 66.5, 64.5, 57.0, 58.33, 56.67, 59.0, 50.0, 47.83, 45.5, 48.83, 48.67, 40.5]


 31%|███▏      | 20/64 [08:04<17:45, 24.22s/it]


KeyboardInterrupt: 

In [24]:
for name, W in grasp_model.named_parameters():
    print(name, W.shape)

rgb_features.0.weight torch.Size([64, 3, 11, 11])
rgb_features.0.bias torch.Size([64])
features.0.weight torch.Size([192, 128, 5, 5])
features.0.bias torch.Size([192])
features.4.weight torch.Size([384, 192, 3, 3])
features.4.bias torch.Size([384])
features.7.weight torch.Size([256, 384, 3, 3])
features.7.bias torch.Size([256])
features.10.weight torch.Size([256, 256, 3, 3])
features.10.bias torch.Size([256])
features.13.weight torch.Size([256, 128, 3, 3])
features.13.bias torch.Size([128])
features.15.weight torch.Size([128, 128, 5, 5])
features.15.bias torch.Size([128])
features.17.weight torch.Size([128, 64, 3, 3])
features.17.bias torch.Size([64])
grasp.0.weight torch.Size([64, 5, 11, 11])
grasp.0.bias torch.Size([5])
confidence.0.weight torch.Size([64, 1, 11, 11])
confidence.0.bias torch.Size([1])


In [26]:
shapley_score_cls = tmc(model=grasp_model, 
                   layer="rgb_features.0.weight",
                   criterion=grasp_criterion, 
                   iterations=10, 
                   vt=20, 
                   verbose=True)

Iteration 1/10:
Dataset size: 400
Training steps: 320 -- Val steps: 80


  0%|          | 0/64 [00:00<?, ?it/s]

Dataset size: 400
Training steps: 320 -- Val steps: 80



  2%|▏         | 1/64 [00:22<23:34, 22.46s/it]

[81.0, 80.5]
Dataset size: 400
Training steps: 320 -- Val steps: 80


...


 69%|██████▉   | 44/64 [17:03<08:03, 24.16s/it]

[80.0, 78.75, 80.5, 80.75, 80.75, 78.5, 78.75, 80.25, 79.0, 83.0, 81.5, 81.25, 79.5, 79.75, 83.75, 80.0, 80.5, 82.5, 81.25, 79.75, 77.5, 78.0, 76.5, 78.0, 75.75, 77.5, 76.0, 78.25, 78.25, 75.0, 75.25, 79.0, 76.25, 76.0, 75.75, 72.0, 68.75, 66.75, 63.0, 61.0, 57.75, 53.5, 54.25, 55.25, 57.25]
Dataset size: 400
Training steps: 320 -- Val steps: 80



 69%|██████▉   | 44/64 [17:22<07:53, 23.69s/it]s]


KeyboardInterrupt: 


 74%|███████▍  | 295/400 [00:32<00:07, 14.30it/s]